<a href="https://colab.research.google.com/github/edwardj4747/Dataset-Labeling/blob/master/ML_notebooks/Prepare_keyword_ML_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import json

In [ ]:
with open('/content/sample_data/KEYWORD_sentences.json') as f:
  raw_data = json.load(f)

Restrict the ground_truths datasets to only ML2xx

In [ ]:
for key, value in raw_data.items():
  new_ground_truths = []
  for gt in value['ground_truths']:
    if gt.startswith("ML2"):
      new_ground_truths.append(gt)
  raw_data[key]['ground_truths'] = new_ground_truths

In [ ]:
count = 0
for k, v in raw_data.items():
  print(k, v)
  count += 1
  if count == 3:
    break

4MFNKR4V {'ground_truths': ['ML2T', 'ML2H2O', 'ML2O3'], 'keyword_sentences': ['t h2o h2o v5 t h2o airs airs', 't h2o t h2o v2 merra-2 t h2o merra-2', 't t t t h2o mls aura aura mls t mls t o3 mls', 'o3 mls mls o3 mls o3 mls h2o mls h2o mls h2o mls h2o airs aqua airs v6 o3 o3 o3 h2o o3 merra-2', 'o3 h2o t h2o airs mls merra v5 merra-2 merra-2 merra merra-2 merra-2', 'merra-2 merra-2 t h2o o3 merra-2 o3 h2o mls t h2o o3 airs mls merra-2', 'merra-2 t o3 merra-2 o3 merra-2 o3 merra-2 h2o h2o merra-2 h2o merra-2 h2o merra-2 airs mls h2o merra-2 h2o', 'merra-2 t merra-2 t merra-2 t t t t merra-2 t t o3 o3 o3 merra-2', 'h2o h2o h2o merra-2 h2o h2o merra-2 h2o t merra-2 merra-2 t merra-2 o3 o3 merra-2 merra-2 h2o', 'h2o merra-2 h2o merra-2 merra-2 t t mls o3 merra-2 o3 merra-2 h2o o3 h2o h2o', 'h2o t t merra-2 t h2o h2o h2o t t t', 't merra-2 o3 h2o t merra-2 t mls o3 o3 merra-2 h2o h2o merra-2 h2o']}
ZFGFZZTV {'ground_truths': ['ML2H2O'], 'keyword_sentences': ['h2o h2o h2o t h2o h2o h2o h2o o

In [ ]:
len(raw_data)

247

In [ ]:
threshold = 10
counts = {}
for key, value in raw_data.items():
  for gt in value['ground_truths']:
    if gt.startswith("ML2"):
      counts[gt] = counts.get(gt, 0) + 1

all_datasets_above_threshold = set()
for key, value in counts.items():
  if value >= threshold:
    all_datasets_above_threshold.add(key)

counts, all_datasets_above_threshold

({'ML2BRO': 3,
  'ML2CH3CL': 4,
  'ML2CH3CN': 2,
  'ML2CH3Cl': 1,
  'ML2CH3OH': 2,
  'ML2CLO': 21,
  'ML2CO': 36,
  'ML2DGG': 1,
  'ML2GPH': 6,
  'ML2H2O': 69,
  'ML2H2O_NRT': 1,
  'ML2HCL': 25,
  'ML2HCN': 3,
  'ML2HCl': 2,
  'ML2HNO3': 39,
  'ML2HNO3_NRT': 1,
  'ML2HO2': 6,
  'ML2HOCL': 3,
  'ML2IWC': 18,
  'ML2N2O': 24,
  'ML2NO2': 1,
  'ML2O3': 115,
  'ML2OH': 9,
  'ML2RHI': 6,
  'ML2SO2': 5,
  'ML2T': 34},
 {'ML2CLO',
  'ML2CO',
  'ML2H2O',
  'ML2HCL',
  'ML2HNO3',
  'ML2IWC',
  'ML2N2O',
  'ML2O3',
  'ML2T'})

In [ ]:
# if want to create a mapping that does not include all datasets
all_datasets = all_datasets_above_threshold

In [ ]:
dataset_mapping = {dataset: i for i, dataset in enumerate(all_datasets)}
dataset_mapping

{'ML2CLO': 3,
 'ML2CO': 7,
 'ML2H2O': 0,
 'ML2HCL': 6,
 'ML2HNO3': 4,
 'ML2IWC': 8,
 'ML2N2O': 2,
 'ML2O3': 1,
 'ML2T': 5}

###### save the dataset mapping

In [ ]:
# save the dataset_mapping
threshold = str(threshold)
dataset_mapping_file_name = "Feb24_" +str(len(dataset_mapping)) + "_dataset_mapping_threshold_" + threshold

with open('ml_ready_data/' + dataset_mapping_file_name + '.json', 'w', encoding='utf-8') as f:
  json.dump(dataset_mapping, f, indent=4)

dataset_mapping_file_name

'Feb24_9_dataset_mapping_threshold_10'

## For each paper create one sentence and the datasets_used vector

In [ ]:
num_papers = len(raw_data)
num_datasets = len(dataset_mapping)
num_papers, num_datasets

(247, 9)

Use all 247 papers, even if they don't have any papers

In [ ]:
papers_sentences = []  # one sentence for each paper
dataset_labels = np.zeros( (num_papers, num_datasets))
dataset_labels_list = []
rows_to_delete = []

for index, (key, value) in enumerate(raw_data.items()):
  if len(value['keyword_sentences']) == 0:
    rows_to_delete.append(index)
    continue
  document_sentences = '. '.join(value['keyword_sentences'])
  papers_sentences.append(document_sentences)

#   # add in the datasets
  for gt in value['ground_truths']:
    if dataset_mapping.get(gt, None) is not None:  # some keys may not have values b/c didn't meet threshold
      dataset_labels[index, dataset_mapping[gt]] = 1

dataset_labels = np.delete(dataset_labels, rows_to_delete, 0)

In [ ]:
len(papers_sentences), dataset_labels.shape

(243, (243, 9))

Create integer encoded words for use with an embedding layer later

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,.:;<=>?@[\\]^_`{|}~\t\n')  # remove / and - 'noaa-18' to noaa18

In [ ]:
tokenizer.fit_on_texts(papers_sentences)

In [ ]:
len(tokenizer.word_index)  # number of unique words

119

In [ ]:
int_encoded_sequences = tokenizer.texts_to_sequences(papers_sentences)
len(int_encoded_sequences)

243

Save the integer encoded representations

In [ ]:
# save the X data
x_file_name = "Feb24_int_encoded_miv_plus_version_243_papers_119_words"
x_file_name

'Feb24_int_encoded_miv_plus_version_243_papers_119_words'

In [ ]:
with open('/content/ml_ready_data/' + x_file_name + '.json', 'w') as f:
  json.dump(int_encoded_sequences, f, indent=4)

Save the dataset and mapping

In [ ]:
# save the dataset_mapping
threshold = str(threshold)
dataset_mapping_file_name = "Feb24_"+ str(len(dataset_mapping)) + "_dataset_mapping_threshold_" + threshold + '_243_papers'

with open('ml_ready_data/' + dataset_mapping_file_name + '.json', 'w', encoding='utf-8') as f:
  json.dump(dataset_mapping, f, indent=4)

dataset_mapping_file_name

'Feb24_9_dataset_mapping_threshold_10_243_papers'

In [ ]:
# save the y data
dataset_labels.shape
y_file_name = "Feb24_ML2xx_num_datasets_" + str(dataset_labels.shape[1]) + "_papers_" + str(dataset_labels.shape[0])
y_file_name

'Feb24_ML2xx_num_datasets_9_papers_243'

In [ ]:
with open('/content/ml_ready_data/' + y_file_name + '.npy', 'wb') as f:
  np.save(f, dataset_labels)